In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [2]:

# Problem Data
I = [0, 1, 2, 3, 4]  # Customers
J = [0, 1, 2]  # Facilities
K = [0, 1, 2] # scenarios

demand = {
    1: [80, 40, 120],
    2: [270, 360, 225],
    3: [250, 200, 275],
    4: [160, 180, 120],
    5: [180, 175, 215],
}

d = np.array(list(demand.values()))

q_p = [22, 18, 15, 20, 24]
q_m = [5, 2, 1, 3, 4]

t = {(0, 0): 4, (1, 0): 5, (2, 0): 6, (3, 0): 8, (4, 0): 10,
     (0, 1): 6, (1, 1): 4, (2, 1): 3, (3, 1): 5, (4, 1): 8,
     (0, 2): 9, (1, 2): 7, (2, 2): 4, (3, 2): 3, (4, 2): 4}

c = [750, 1250, 1000]
v_ = [5, 7, 6]
M = [500, 500, 500]

In [3]:
# Extensive Form
m = gp.Model()

x = m.addVars(J, vtype=GRB.BINARY, name="x")
y = m.addVars(I, J, lb=0.0, name="y")
w_p = m.addVars(I, K,  lb=0.0, name="w_plus")
w_m = m.addVars(I, K, lb=0.0, name="w_minus")

capacity_constraint = m.addConstrs((gp.quicksum(y[i, j] for i in I) <= M[j] * x[j] for j in J), name="capacity")
w_constraint = m.addConstrs((w_p[i, k] - w_m[i, k] == d[i, k] - gp.quicksum(y[i, j] for j in J) for i in I for k in K), name="w constraint")


m.setObjective(gp.quicksum(c[j] * x[j] for j in J) - gp.quicksum((v_[j] - t[i, j])*y[i, j] for i in I for j in J) - (1/3)*gp.quicksum(-q_p[i]*w_p[i, k] - q_m[i]*w_m[i, k] for i in I for k in K), GRB.MINIMIZE)
m.update()
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-01-17
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 18 rows, 48 columns and 93 nonzeros
Model fingerprint: 0x7c9a4c99
Variable types: 45 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [3e-01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 4e+02]
Presolve removed 10 rows and 11 columns
Presolve time: 0.00s
Presolved: 8 rows, 37 columns, 52 nonzeros
Variable types: 35 continuous, 2 integer (2 binary)
Found heuristic solution: objective 2378.3333333

Root relaxation: objective 6.833333e+01, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     W

In [20]:
# Start an environment
ENV = gp.Env(empty=True)
ENV.setParam("OutputFlag", 0)
ENV.start()

# multi-cut integer l-shaped method

print("Running Multi-cut Integer L-Shaped Method")
rmp = gp.Model(env=ENV)
x = rmp.addVars(J, vtype=GRB.BINARY, name="x")
y = rmp.addVars(I, J, lb=0.0, name="y")

# multicut theta
theta = rmp.addVars(K, lb=-float("inf"), name="theta")

cap = rmp.addConstrs((gp.quicksum(y[i, j] for i in I) <= M[j] * x[j] for j in J), name="capacity")

rmp.setObjective(gp.quicksum(c[j]*x[j] for j in J) - gp.quicksum((v_[j] - t[i, j])*y[i, j] for i in I for j in J) + gp.quicksum(theta[k] for k in K), GRB.MINIMIZE)


optimal = False
r = s = v = 0

for _ in range(200):  # max iters
    print(f"r = {r}; s={s}; v={v}")
    v += 1
    rmp.update()
    rmp.optimize()

    # get rmp solution
    y_v = np.zeros((len(I), len(J)))
    for i, j in y:
        y_v[i, j] = y[i, j].X  #

    theta_v = np.array([theta[k].X for k in theta])

    fc = False  # use this to indicate whether a feasibility cut has been added; true --> no fc, proceed to oc
    for k in K:
        print("feasibility iteration")
        fcp = gp.Model(env=ENV)

        w_p = fcp.addVars(I,  lb=0.0, name="w_plus")
        w_m = fcp.addVars(I, lb=0.0, name="w_minus")

        vp = fcp.addVars(I, lb=0.0, name="v+")
        vm = fcp.addVars(I, lb=0.0, name="v-")

        w_fcp = fcp.addConstrs((w_p[i] - w_m[i] + vp[i] - vm[i] == d[i, k] - np.sum(y_v[i]) for i in I), name="w constraint")
        fcp.update()

        fcp.setObjective(gp.quicksum(vp[i] for i in I) + gp.quicksum(vm[i] for i in I), GRB.MINIMIZE)
        fcp.optimize()

        if fcp.ObjVal > 0:
            sig_v = np.array([w_fcp[i].Pi for i in w_fcp])
            dr = sig_v @ d[:, k]

            rmp.addConstr(gp.quicksum(sig_v[i] * gp.quicksum(y[i, j] for j in J) for i in I) >= dr, name=f"fc_{r+1}")

            rmp.addConstrs((sig_v[i] * gp.quicksum(y[i, j] for j in J) >= dr for i in I))
            r += 1
            break
        else:
            fc = True

    if fc:
        print("generating optimality cuts")
        oc = False
        for k in K:
            ocp = gp.Model(env=ENV)
            # ocp.setParam('DualReductions', 0)
            w_p = ocp.addVars(I, lb=0.0, name="w_plus")
            w_m = ocp.addVars(I, lb=0.0, name="w_minus")

            w_ocp = ocp.addConstrs((w_p[i] - w_m[i] == d[i, k] - np.sum(y_v[i]) for i in I), name="w constraint")
            ocp.update()

            ocp.setObjective(gp.quicksum(q_p[i]*w_p[i] + q_m[i]*w_m[i] for i in I), GRB.MINIMIZE)
            ocp.optimize()

            pi_v = np.array([w_ocp[i].Pi for i in I])

            if theta_v[k] < (1/3) * (np.dot(pi_v, d[:, k] - np.sum(y_v, axis=1))):
                oc = True
                es = (1/3) * np.dot(pi_v, d[:, k])
                rmp.addConstr(theta[k] >= (1/3) * gp.quicksum(pi_v[i] * (gp.quicksum(y[i, j] for j in J) - d[i, k]) for i in I), name=f"oc_{s+1}")
                s += 1

        if oc:
            continue # back to step 1
        else:
            print("Optimal Soln Found")
            optimal = True # problem is optimal
    else:
        continue  # back to step 1

    if optimal:
        break

Running Multi-cut Integer L-Shaped Method
r = 0; s=0; v=0
feasibility iteration
feasibility iteration
feasibility iteration
generating optimality cuts
r = 0; s=3; v=1
feasibility iteration
feasibility iteration
feasibility iteration
generating optimality cuts
r = 0; s=6; v=2
feasibility iteration
feasibility iteration
feasibility iteration
generating optimality cuts
r = 0; s=9; v=3
feasibility iteration
feasibility iteration
feasibility iteration
generating optimality cuts
r = 0; s=12; v=4
feasibility iteration
feasibility iteration
feasibility iteration
generating optimality cuts
r = 0; s=15; v=5
feasibility iteration
feasibility iteration
feasibility iteration
generating optimality cuts
r = 0; s=18; v=6
feasibility iteration
feasibility iteration
feasibility iteration
generating optimality cuts
r = 0; s=21; v=7
feasibility iteration
feasibility iteration
feasibility iteration
generating optimality cuts
r = 0; s=24; v=8
feasibility iteration
feasibility iteration
feasibility iteration